#### 1. spark如何实现容错性?
spark的容错性有2个手段, 检查点和RDD血统
* 检查点: checkpoint, 将RDD数据持久化到hdfs中, 并打断rdd的linage
* RDD血统: 每个RDD都记录有自己的血统, 一个计算序列, 一旦有哪个RDD分区丢失或其它问题, 可以根据自身的RDD重新计算,   
   重算可以在某个几点分区下执行, 不会导致系统回滚


#### 2. spark和hadoop的mr有什么区别
首先, 两者都是基于MR的计算模型, 但有很多不同
* hadoop的一个job, 只有map操作和reduce操作, 多数据的表达能力欠缺, 每次reduce后要将数据存到hdfs上; 为了完成一个目标往往要串行执行多个job, 造成大量的HDFS IO浪费   
* spark将1个job划分为多个stage, stage的划分是通过RDD之间的依赖关系判定的; 每个stage产生的结果会存在内存中, 避免多个stage串行的情况下对hdfs的读写请求. 而且spark拥有很多算子, 数据表现力上很强

#### 3. RDD中的宽窄依赖? 有什么作用?
* 窄依赖是指父RDD的每个分区都只被子RDD的一个分区所使用。相应的，那么宽依赖就是指父RDD的分区被多个子RDD的分区所依赖。  
  例如，map就是一种窄依赖，而join则会导致宽依赖，主要是看有没有shuffle操作。
* 宽窄依赖的作用是用来划分stage。

#### 4. cache和persist的区别？
它们都是用来进行缓存RDD的。
* cache是特定的persist，rdd中cache的缓存级别是MEMORY_ONLY，cache调用了persist；
* persist可以设置不同的缓存级别。  

一共有几种缓存级别
* MEMORY_ONLY：数据保存在内存中，如果内存不够，数据可能就不会持久化；
* MEMORY_AND_DISK：数据优先保存在内存中，如果内存不够则会存到磁盘中；
* MEMORY_ONLY_SER：和MEMORY_ONLY类似，区别是会将RDD中的数据进行序列化，这种方式更加节省内存；
* MEMORY_AND_DISK_SER：和MEMORY_AND_DISK类似，区别是会将RDD中的数据进行序列化，这种方式更加节省内存；
* DISK_ONLY：将数据全部写入磁盘文件中；
* MEMORY_ONLY_2, MEMORY_AND_DISK_2, 等等：这种有后缀_2的，代表的是将每个持久化的数据，都复制一份副本，并将副本保存到其他节点上。这种基于副本的持久化机制主要用于进行容错。

#### 5. 广播变量和累加器? 
* **广播变量**  
广播变量是把一个集合, 通过driver节点发给每个executor, executor只能使用这个集合但不能修改这个集合. 使用广播变量时, driver要先把一个RDD进行collect(), 然后再广播发送出去, 因此, driver的网络压力就会很大
* **累加器**   
累加器是在Driver端记录的一个全局值, 在driver声明, 在executor端更新, 且只能触发add()方法, 最终由driver端调用collect得到最终结果

#### 6. groupByKey和reduceByKey哪个好
reduceByKey更好  
* reduceByKey中指定的聚合函数, 会在map端先进性聚合, 再发往reducer进行计算;   
* 而groupByKey直接将数据发到reducer, 网络通信量会大很多

### 7. kafka
* **Kafka 分布式的情况下，如何保证消息的顺序?**
  kafka分布式情况下, 数据只能保证局部有序, 不能保证全局有序. 什么意思?
    * kafka是划分partition的, 一个partition因为被一个write agead log记录, 因此同一个partition下的数据可以有序; 
    * 不同partition之间, 不能保证顺序。  
      不过绝大多数用户都可以通过 message key 来定义，因为同一个 key 的 message 可以保证只发送到同一个 Partition。  
      比如说 key 是 user id，table row id 等等，所以同一个 user 或者同一个 record 的消息永远只会发送到同一个 Partition上，保证了同一个 user 或 record 的顺序。
      
      
* **Spark streaming 读取kafka数据的两种方式?**   
    1. 基于Receiver方式   
       需要使用单独的Receiver线程来异步获取Kafka数据。Spark Streaming启动时，会在Executor中同时启动Receiver异步线程用于从Kafka持续获取数据，获取的数据先存储在Receiver中(存储方式由StorageLevel决定)，后续，当Batch Job触发后，这些数据会被转移到剩下的Executor中被处理。处理完毕后，Receiver会自动更新Zookeeper中的Offset。
    2. 基于Direct(No Receiver)方式    
       不需要使用单独的Receiver线程从Kafka获取数据。Spark Streaming Batch Job触发时，Driver端确定要读取的Topic-Partition的OffsetRange，然后由Executor并行从Kafka各Partition读取数据并计算。

### 8. Spark处理数据倾斜有什么好方法
简单一句: Spark 数据倾斜的几种场景以及对应的解决方案，包括避免数据源倾斜，调整并行度，使用自定义 Partitioner，使用 Map 侧 Join 代替 Reduce 侧 Join（内存表合并），给倾斜 Key 加上随机前缀等。下面分情况讨论: 
* **如果是多个不同的key落在一个分区导致的数据倾斜**  
    * 可以调大并行度: 
    ```
    spark.default.parallelism : rdd分区数
    spark.sql.shuffle.partitions: SparkSQL通过通过这个值修改并发数。
    ```
    * 或者可以自定义Partitioner
    
    
* **如果是join产生的数据倾斜, 且发现其中有一个表很小**
    * 可以将小表通过broadcast广播出去  
      这样完全避免了shuffle, 让join完全变成maper端的join
    * 或者可以把大表的key增加随机前缀, 小表扩大n倍进行join, 最后再删除key的前缀即可
      
      
* **如果是1个非常非常大的key导致的数据倾斜**  
    * 可以将这个key拿出来单独处理, 给这个key加上随机前缀, 打散这个key

[spark面试题1](https://zhuanlan.zhihu.com/p/49169166)   
[整理spark shuffle](https://tech.youzan.com/spark_memory_1/)